# Morris Screening

In this notebook we apply the popular Morris screening method to a building design problem.
We determine the sensitivty of the objective (electricty use) to each of the design parameters.

In [ ]:
import numpy as np
import pandas as pd
from besos import eppy_funcs as ef, sampling
from besos.evaluator import EvaluatorEP
from besos.problem import EPProblem

from SALib.analyze import morris as manalysis
from SALib.sample import morris as msampling
from parameter_sets import parameter_set

## Build an EnergyPlus Evaluator

In [ ]:
parameters = parameter_set(7)  # use a pre-defined parameter set
problem = EPProblem(parameters, ["Electricity:Facility"])
building = ef.get_building()  # use the example building
evaluator = EvaluatorEP(problem, building)
inputs = sampling.dist_sampler(
    sampling.lhs, problem, 50
)  # get 50 samples of the input space

## Conduct a Morris screening of the parameters

The following cells conduct a Morris screening, a global sensitivity method.
It uses $r$ times $n$ one-at-time changes (OAT) of each parameter at randomly selected points.
The resulting distribution of $r$ samples provides a mean $\mu^*$, and a standard deviation $\sigma$ of the elementary effects of the $i$-th input parameter. [[1]] [[2]] [[3]]

[1]: https://www.sciencedirect.com/science/article/pii/S1364032112007101
[2]: https://en.wikipedia.org/wiki/Elementary_effects_method
[3]: https://en.wikipedia.org/wiki/Morris_method

In [ ]:
names = [parameters[i].name for i in range(len(parameters))]
bounds = [
    [parameters[i].value_descriptor.min, parameters[i].value_descriptor.max]
    for i in range(len(parameters))
]

problem = {"num_vars": len(parameters), "names": names, "bounds": bounds}

X = np.round(msampling.sample(problem, N=5), decimals=3)
inputs = pd.DataFrame(data=X, columns=names)
outputs = evaluator.df_apply(inputs)


Y = outputs.values
Si = manalysis.analyze(
    problem, X, Y, conf_level=0.95, print_to_console=True, num_levels=4
)
pd.DataFrame(data=Si["mu_star"], index=Si["names"]).sort_values(by=0)